<a href="https://colab.research.google.com/github/manisharavindh/basic-pytorch-models/blob/main/notebooks/01_linear_regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

print(torch.__version__)

2.6.0+cu124
